In [1]:
import pprint
import os

# Set the path before importing HF libraries
os.environ["HF_HOME"] = "/media/aiseed/AISeed"

In [2]:
import torch
from typing import Optional, Dict, Any

if torch.cuda.is_available():
    device = "cuda"
    print(f"Using CUDA GPU: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = "mps"
    print("Using Apple MPS")
else:
    device = "cpu"
    print("Using CPU - you will need to use a GPU to train models")

Using CUDA GPU: NVIDIA GeForce RTX 4070 SUPER
GPU memory: 12.4GB


In [3]:
# Import required libraries for fine-tuning
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import torch
# import wandb  # Optional: for experiment tracking

# Initialize Weights & Biases (optional)
# wandb.init(project="smollm3-finetuning")

# Load SmolLM3 base model for fine-tuning
model_name = "HuggingFaceTB/SmolLM3-3B-Base"
new_model_name = "SmolLM3-Custom-SFT"

print(f"Loading {model_name}...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
tokenizer.padding_side = "right"  # Padding on the right for generation

print(f"Model loaded! Parameters: {model.num_parameters():,}")

Loading HuggingFaceTB/SmolLM3-3B-Base...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded! Parameters: 3,075,098,624


In [4]:

instruct_model_name = "HuggingFaceTB/SmolLM3-3B"

# Load tokenizers
instruct_tokenizer = AutoTokenizer.from_pretrained(instruct_model_name)


In [5]:
# Load and prepare training dataset
print("=== PREPARING DATASET ===\n")

# Option 1: Use SmolTalk2 (recommended for beginners)
dataset = load_dataset("HuggingFaceTB/smoltalk2", "SFT")
train_dataset = dataset["smoltalk_everyday_convs_reasoning_Qwen3_32B_think"].select(range(1000))  # Use subset for faster training

# Option 2: Use your own processed dataset from Exercise 2
# train_dataset = gsm8k_formatted.select(range(500))

print(f"Training examples: {len(train_dataset)}")
print(f"Example: {train_dataset[0]}")

# Prepare the dataset for SFT
def format_chat_template(example):
    """Format the messages using the chat template"""
    if "messages" in example:
        # SmolTalk2 format
        messages = example["messages"]
    else:
        # Custom format - adapt as needed
        messages = [
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["response"]}
        ]
    
    # Apply chat template
    text = instruct_tokenizer.apply_chat_template(
        messages, 
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Apply formatting
formatted_dataset = train_dataset.map(format_chat_template)
formatted_dataset = formatted_dataset.remove_columns(
    [col for col in formatted_dataset.column_names if col != "text"]
)
print(f"Formatted example: {formatted_dataset[0]['text'][:200]}...")

=== PREPARING DATASET ===



Resolving data files:   0%|          | 0/124 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/113 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/113 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/105 [00:00<?, ?it/s]

Training examples: 1000
Example: {'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': '<think>\nOkay, the user sent "Hi there". That\'s a friendly greeting. I should respond in a welcoming way. Let me check the guidelines. I need to be helpful, keep the conversation going, and maybe ask how I can assist them. Let me make sure the tone is warm and approachable. Alright, something like "Hello! How can I assist you today?" That should work. Let me confirm there\'s no typo and it\'s in a natural, conversational style.\n</think>\n\nHello! How can I assist you today?', 'role': 'assistant'}, {'content': "I'm looking for a healthy breakfast idea. What's a good option?", 'role': 'user'}, {'content': "<think>\nOkay, the user is asking for a healthy breakfast idea. Let me think about what makes a breakfast healthy. It should be balanced, providing a mix of nutrients like protein, fiber, healthy fats, and some carbs. Let me brainstorm some options.\n\nMaybe start with a classic like 

In [13]:
# Configure training parameters
training_config = SFTConfig(
    # Model and data
    output_dir=f"./{new_model_name}",
    dataset_text_field="text",
    max_length=1024,
    
    # Training hyperparameters
    per_device_train_batch_size=1,  # Adjust based on your GPU memory
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    num_train_epochs=1,  # Start with 1 epoch
    max_steps=500,  # Limit steps for demo
    
    # Optimization
    warmup_steps=50,
    weight_decay=0.01,
    optim="adamw_torch",
      
    # Logging and saving
    logging_steps=2,
    save_steps=100,
    eval_steps=100,
    save_total_limit=2,
    
    # Memory optimization
    dataloader_num_workers=0,
    group_by_length=True,  # Group similar length sequences
    
    # Hugging Face Hub integration
    push_to_hub=False,  # Set to True to upload to Hub
    # hub_model_id=f"your-username/{new_model_name}",
    
    # Experiment tracking
    report_to=["trackio"],  # Use trackio for experiment tracking
    run_name="finetune-smol-lm3-2",
)

print("Training configuration set!")
print(f"Effective batch size: {training_config.per_device_train_batch_size * training_config.gradient_accumulation_steps}")

Training configuration set!
Effective batch size: 1


In [ ]:
# LoRA configuration with PEFT
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
)

# Create SFTTrainer with LoRA enabled




/media/aiseed/AISeed/code/huggingface-collection/.venv/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/media/aiseed/AISeed/code/huggingface-collection/.venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [14]:
from trl import SFTTrainer

lora_trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,  # dataset with a "text" field or messages + dataset_text_field in config
    args=training_config,
    peft_config=peft_config,  # << enable LoRA
)

/media/aiseed/AISeed/code/huggingface-collection/.venv/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/media/aiseed/AISeed/code/huggingface-collection/.venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [15]:
print("Starting LoRA training…")
lora_trainer.train()

Starting LoRA training…
* Created new run: finetune-smol-lm3-2


Step,Training Loss
2,1.988000
4,1.862900
6,1.874600
8,1.942200
10,1.863700
12,1.903100
14,1.809800
16,1.972700
18,1.869000
20,1.841400


* Run finished. Uploading logs to Trackio (please wait...)


TrainOutput(global_step=500, training_loss=1.010246517419815, metrics={'train_runtime': 567.9924, 'train_samples_per_second': 0.88, 'train_steps_per_second': 0.88, 'total_flos': 8503544629481472.0, 'train_loss': 1.010246517419815, 'epoch': 0.5})